### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M12D13', 'UTC 16:28')


# Settings

In [2]:
MAXTASKS = 4
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,id,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,27.502333,1513180813664,global_historical_WS_year_dimensionless_30sPfa...,VIOVH45IVP42VTWJAAZAFLC2,0.0,0.026533,27.528867,16:00:15,1.513181e+12,RUNNING,EXPORT_IMAGE,1513182465396
1,27.502400,1513180811196,global_historical_WS_year_dimensionless_30sPfa...,JT45TTFSP4ULLGZ5RSIQMAX6,0.0,0.060200,27.562600,16:00:14,1.513181e+12,RUNNING,EXPORT_IMAGE,1513182464952
2,27.502483,1513180808749,global_historical_WS_year_dimensionless_30sPfa...,KDV7Z4BRIF7VZ2T3LSI73K4V,0.0,0.053117,27.555600,16:00:11,1.513181e+12,RUNNING,EXPORT_IMAGE,1513182462085
3,NaN,1513180551271,global_historical_WS_month_dimensionless_30sPf...,DJBECBAOVJQQ33VYBK2A4OTG,NaN,NaN,NaN,NaN,NaN,CANCELLED,EXPORT_IMAGE,1513180704816


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)